In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import genomepy

import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

In [6]:
import celloracle as co
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
co.__version__


'0.18.0'

In [7]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

In [8]:
genomepy.install_genome(name="hg38", provider="UCSC")

13:29:35 | INFO | Downloading assembly summaries from UCSC
13:29:40 | INFO | Downloading genome from UCSC. Target URL: https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz...


Download:   0%|          | 0.00/938M [00:00<?, ?B/s]

13:31:10 | INFO | Genome download successful, starting post processing...
13:31:27 | INFO | name: hg38
13:31:27 | INFO | local name: hg38
13:31:27 | INFO | fasta: /home/uvictor/.local/share/genomes/hg38/hg38.fa


Filtering Fasta: 0.00 lines [00:00, ? lines/s]

Fasta("/home/uvictor/.local/share/genomes/hg38/hg38.fa")

In [9]:
ref_genome = "hg38"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome,
                                             genomes_dir=None)

print(ref_genome, "installation: ", genome_installation)

hg38 installation:  True


In [10]:
peaks = pd.read_csv("/mnt/c/Users/UVictor/Documents/scATAC_Output/processed_peak_file.csv", index_col=0)
peaks.head()

,peak_id,gene_short_name
0,chr10_100009705_100010205,DNMBP
1,chr10_100010297_100010797,DNMBP
2,chr10_100185807_100186307,ERLIN1
3,chr10_100186405_100186905,ERLIN1
4,chr10_100229354_100229854,CHUK


In [11]:
peaks = ma.check_peak_format(peaks, ref_genome, genomes_dir=None)

Peaks before filtering:  37077
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  37077


In [12]:
tfi = ma.TFinfo(peak_data_frame=peaks,
                ref_genome=ref_genome,
                genomes_dir=None)

In [13]:
tfi.scan(fpr=0.02,
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path="/mnt/c/Users/UVictor/Documents/scATAC_Output/TFIScanned.celloracle.tfinfo")

No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2024-10-21 14:15:19,177 - DEBUG - using background: genome hg38 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 



2024-10-21 14:15:25,820 - DEBUG - determining FPR-based threshold


Motif scan started .. It may take long time.



Scanning:   0%|          | 0/35174 [00:00<?, ? sequences/s]

In [14]:
tfi.scanned_df.head()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100009705_100010205,GM.5.0.Mixed.0001,,"SRF, EGR1",8.578792,91,-1
1,chr10_100009705_100010205,GM.5.0.Mixed.0001,,"SRF, EGR1",8.319869,350,-1
2,chr10_100009705_100010205,GM.5.0.Mixed.0001,,"SRF, EGR1",7.919846,458,-1
3,chr10_100009705_100010205,GM.5.0.Mixed.0001,,"SRF, EGR1",7.680395,459,-1
4,chr10_100009705_100010205,GM.5.0.Mixed.0001,,"SRF, EGR1",7.575280,87,-1


In [15]:
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

Filtering finished: 9850440 -> 2108806
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/35174 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/20169 [00:00<?, ?it/s]

  0%|          | 0/1094 [00:00<?, ?it/s]

In [16]:
df = tfi.to_dataframe()
df.head()

,peak_id,gene_short_name,9430076C15RIK,AC002126.6,AC012531.1,AC226150.2,AFP,AHR,AHRR,AIRE,...,ZNF784,ZNF8,ZNF816,ZNF85,ZSCAN10,ZSCAN16,ZSCAN22,ZSCAN26,ZSCAN31,ZSCAN4
0,chr10_100009705_100010205,DNMBP,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr10_100010297_100010797,DNMBP,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr10_100185807_100186307,ERLIN1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr10_100186405_100186905,ERLIN1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr10_100229354_100229854,CHUK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#df = tfi.to_dataframe()
df.to_parquet("/mnt/c/Users/UVictor/Documents/scATAC_Output/Greenleaf23_Skin_GRN_dataframe.parquet")